In [6]:
%pip install -U \
    torch==2.0.1 \
    transformers==4.34.2 \
    langchain==0.0.309 \
    langchain_experimental \
    iprogress==0.4

Note: you may need to restart the kernel to use updated packages.


Load libraries: 

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, pipeline

from langchain.llms.base import LLM
from langchain import PromptTemplate
from langchain.llms import HuggingFacePipeline

from langchain_experimental.pal_chain.base import PALChain
import torch

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ModuleNotFoundError: No module named 'langchain_experimental'

Grab a LLM.  We use Hugging Face, and setup a simple pipeline.

In [4]:
model_id = "bigscience/bloomz-3b"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_id, model_max_length=256)
model = AutoModelForCausalLM.from_pretrained(model_id)

In [6]:
pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, temperature=0.8, max_new_tokens=100
)

llm = HuggingFacePipeline(pipeline=pipe)

In [8]:
from langchain import PALChain

pal_chain = PALChain.from_math_prompt(
    llm, 
    verbose=True,
)

pal_template = """Translate a math problem into a expression that can be executed using Python's numexpr library. 
Use the output of running this code to answer the question.

Question: ${{Question with hard calculation.}}
${{Code that prints what you need to know}}


Question: I have three apples and get given two more, how many apples do I have?
def solution():
    initial_apples = 3
    extra_apples = 2
    return initial_apples + extra_apples


Question: Jan has three times the number of pets as Marcia. Marcia has six pets, how many pets does jan have?
def solution():
    marcia_pets = 6
    jan_pets = marcia_pets * 3
    return jan_pets

Question: {question}
"""

pal_prompt = PromptTemplate(
    input_variables=["question"], 
    template=pal_template
)

pal_chain.prompt = pal_prompt

In [9]:
answer = pal_chain.run("Jane has 10 sweets. Peter has twice the number of sweets than Jane. How many sweets does Peter have?")
print(answer)

Token indices sequence length is longer than the specified maximum sequence length for this model (1072 > 256). Running this sequence through the model will result in indexing errors




> Entering new PALChain chain...
def solution():
    """Jane has 10 sweets. Peter has twice the number of sweets than Jane. How many sweets does Peter have?"""
    sweets_initial = 10
    sweets_peter = 10
    sweets_jane = sweets_initial * 2
    result = sweets_jane
    return result

> Finished chain.
20


This is a simple example entirely running inside this notebook.  With a larger model we would be able to achive more sophisticated output.  In this case, we could have called the LLM directly with our prompt, and gotten the same answer. 

In [10]:
llm("Peter has twice the number of sweets than Jane.  Jane has 10 sweets.  How many sweets does Peter have?")

' 20'